## Store Manager Dashboard

Queries that would be run to create the different values that we would need to create a dashboard for the store manager.

From the data we have, it would be important for the store manager to know about inventory levels, how films are being returned and potentially risky customers.

We will write scripts to perform the following operations.

- Look at the stock level for each film
    
- <span style="color: #008000;">Compare inventory level of total rentals for each movie</span>
    
- Look for repeat overdue offenders
    
- See if repeat overdue offenders are also don't return books
    
- How often are non-returners first time customers

In [2]:
-- Look at the inventory level for each film 
CREATE VIEW total_inventory_per_film AS 
SELECT f.title, COUNT(i.inventory_id) AS total_stock 
FROM film AS f 
LEFT JOIN inventory AS i ON f.film_id = i.film_id -- LEFT JOIN because we also want movies that don't appear in the inventory 
GROUP BY f.title
ORDER BY f.title;

SELECT *
FROM total_inventory_per_film;

: relation "film" does not exist

Total execution time: 00:00:00.002

In [2]:
-- Compare inventory level of total rentals for each movie 
CREATE VIEW total_inventory_and_total_rental_per_film AS
SELECT f.title, COUNT(DISTINCT(i.inventory_id, i.film_id)) AS total_inventory, COUNT(DISTINCT(r.inventory_id, r.rental_id)) AS total_rented_out
FROM film AS f
LEFT JOIN inventory AS i ON f.film_id = i.film_id
LEFT JOIN rental AS r ON i.inventory_id = r.inventory_id
GROUP BY f.title
ORDER BY total_rented_out DESC;

SELECT *
FROM total_inventory_and_total_rental_per_film;

CREATE VIEW

Total execution time: 00:00:00.005

In [3]:
-- Looking at the spread of books that were returned on time, not returned or overdue
CREATE VIEW return_window_spread AS
WITH return_periods (rental_duration, rental_period) 
AS (
    SELECT rental_duration, date_part('day', return_date - rental_date) AS rental_period
    FROM film AS f
    INNER JOIN inventory AS i ON f.film_id = i.film_id
    INNER JOIN rental AS r ON r.inventory_id = i.inventory_id)

SELECT return_status, COUNT(*) AS total
FROM (
    SELECT CASE 
                WHEN rental_period > rental_duration THEN 'OVERDUE'
                WHEN rental_period < rental_duration THEN 'ON TIME'
                ELSE 'NOT RETURNED'
            END
            AS return_status
    FROM return_periods
) AS rs
GROUP BY return_status;

SELECT * 
FROM return_window_spread;

CREATE VIEW

Total execution time: 00:00:00.020

In [5]:
-- Create view for getting rental period information
CREATE VIEW repeat_overdue_offenders AS
WITH return_window (customer_id, inventory_id, rental_id, film_id, rental_days_left) AS (
SELECT r.customer_id, i.inventory_id, r.rental_id, f.film_id, f.rental_duration - date_part('day', r.return_date - r.rental_date) AS rental_days_left
FROM film AS f
INNER JOIN inventory AS i ON f.film_id = i.film_id
INNER JOIN rental AS r ON r.inventory_id = i.inventory_id)

-- Look for repeat overdue offenders
SELECT CONCAT(c.first_name, ' ', c.last_name) AS full_name, COUNT(*) AS total_overdue_rentals
FROM return_window AS rw
INNER JOIN customer c ON c.customer_id = rw.customer_id
WHERE rental_days_left < 0
GROUP BY CONCAT(c.first_name, ' ', c.last_name)
ORDER BY total_overdue_rentals DESC;

CREATE VIEW

Total execution time: 00:00:00.005

In [10]:
-- Check if constent over due renters also don't return films 

CREATE VIEW customers_with_overdue_and_no_return AS
WITH 
return_window (customer_id, inventory_id, rental_id, film_id, rental_days_left) 
AS (
    SELECT r.customer_id, i.inventory_id, r.rental_id, f.film_id, f.rental_duration - date_part('day', r.return_date - r.rental_date) AS rental_days_left
    FROM film AS f
    INNER JOIN inventory AS i ON f.film_id = i.film_id
    INNER JOIN rental AS r ON r.inventory_id = i.inventory_id),
customer_overdue_count (customer_id, full_name, total_overdue_rentals)
AS (
    SELECT c.customer_id, CONCAT(c.first_name, ' ', c.last_name) AS full_name, COUNT(*) AS total_overdue_rentals
    FROM return_window AS rw
    INNER JOIN customer c ON c.customer_id = rw.customer_id
    WHERE rental_days_left < 0
    GROUP BY CONCAT(c.first_name, ' ', c.last_name), c.customer_id
    ORDER BY total_overdue_rentals DESC
) 

-- Do people who have more overdue retnals also not bring films back
SELECT rw.customer_id, full_name, total_overdue_rentals, COUNT(*) AS total_non_returnals
FROM customer_overdue_count AS co
INNER JOIN return_window AS rw ON co.customer_id = rw.customer_id
WHERE rw.rental_days_left IS NULL
GROUP BY rw.customer_id, full_name, total_overdue_rentals
ORDER BY total_overdue_rentals DESC;

CREATE VIEW

Total execution time: 00:00:00.007

## Finance Dashboard

These are some queries that can be used when creating a dashboard for the finance team of our rental store.

Here is some information that we can obtain from the database

- What is the total value of our inventory
- What is the total value of the inventory excluding films that have not been returned
- How much is required to replace the movies that have not been returned
- How much money have we made
- Which month is our best month for rentals
- How much have we lost from people not paying for rentals

In [16]:
-- What is the total value of our inventory 
SELECT ROUND(CAST(SUM(replacement_cost) AS NUMERIC), 2) AS total_inventory_value 
FROM inventory AS i 
INNER JOIN film AS f ON f.film_id = i.film_id

-- This output also includes films that have not been returned

(1 row(s) affected)

Total execution time: 00:00:00.004

total_inventory_value
92621.19


In [22]:
-- What is the value of the inventory excluding films that have not been returned 
SELECT ROUND(CAST(SUM(f.replacement_cost) AS NUMERIC), 2) As total_cost
FROM film AS f 
INNER JOIN inventory AS i
ON i.film_id = f.film_id
WHERE inventory_id NOT IN (
    SELECT inventory_id 
    FROM rental
    WHERE return_date IS NULL
)

(1 row(s) affected)

Total execution time: 00:00:00.007

total_cost
89069.02


In [28]:
-- How much is required to replace the movies that have not been returned 
WITH total_inventory_value (excluding_missing, including_missing) AS( -- USING A CTE
    SELECT excluding_missing, including_missing
    FROM ( -- USING A SUB-QUERY
        SELECT ROUND(CAST(SUM(f.replacement_cost) AS NUMERIC), 2) As excluding_missing
        FROM film AS f 
        INNER JOIN inventory AS i
        ON i.film_id = f.film_id
        WHERE inventory_id NOT IN (
            SELECT inventory_id 
            FROM rental
            WHERE return_date IS NULL
        )) AS excluding_missing_films,  -- USING A SUB-QUERY
        (
            SELECT ROUND(CAST(SUM(replacement_cost) AS NUMERIC), 2) AS including_missing
            FROM inventory AS i 
            INNER JOIN film AS f ON f.film_id = i.film_id
        ) AS including_missing_films -- USING A SUB-QUERY
) -- END OF CTE

SELECT (including_missing - excluding_missing) AS cost_for_missing_films
FROM total_inventory_value;


DROP VIEW

CREATE VIEW

(1 row(s) affected)

Total execution time: 00:00:00.017

cost_for_missing_films
3552.17


In [30]:
-- How much money has been made 
SELECT ROUND(CAST(SUM(amount) AS NUMERIC), 2) AS total_earnings
FROM payment 

(1 row(s) affected)

Total execution time: 00:00:00.006

total_earnings
67416.51


In [1]:
-- Which month is the best for rentals and how much is made in each month 
CREATE VIEW best_rental_month AS
SELECT date_part('month', rental_date) AS month, ROUND(CAST(SUM(p.amount) AS NUMERIC), 2) AS total_earnings, COUNT(*) total_rentals
FROM rental AS r
INNER JOIN payment AS p ON r.rental_id = p.rental_id
GROUP BY date_part('month', rental_date)
ORDER BY month;

CREATE VIEW

Total execution time: 00:00:00.005